In [4]:
from ipyleaflet import Map, Marker, Rectangle, AwesomeIcon, FullScreenControl
from ipywidgets import HTML
import overpass

In [5]:
latitude = 35.16597954399553
longitude = 136.89914184408428
zoom = 15

group_name = "fuel"
oapi = overpass.API()


def marker_callback(event, location):
    """ marker位置に応じて rectのboundsも更新 """
    m.remove_layer(rect)
    rect.bounds = gen_bounds(*location)
    m.add_layer(rect)


def remove_layers(m, group_name):
    """ layerから特定のタグが付いたオブジェクトを削除 """
    for l in m.layers:
        if hasattr(l, "alt") and l.alt == group_name:
            m.remove_layer(l)


def search_callback(change):
    """ 更新されたbounds内のfuelを検索して更新 """
    bounds = change["new"]
    update_fuels(m, bounds, group_name)


def gen_bounds(center_lat, center_lon, d_lat=0.01, d_lon=0.015):
    """ 矩形境界を生成 """
    bottom = center_lat - d_lat / 2
    left = center_lon - d_lon / 2
    top = bottom + d_lat
    right = left + d_lon

    return ((bottom, left), (top, right))


def update_fuels(m, bounds, group_name):
    """ bounds内のfuelを検索してMapオブジェクトに追加 """

    query = "(node[amenity=fuel]({0}, {1}, {2}, {3}););".format(
        *bounds[0], *bounds[1])

    response_fuel = oapi.get(query, verbosity='geom')
    remove_layers(m, group_name)

    for features in response_fuel["features"]:
        marker = Marker(location=features["geometry"]["coordinates"][::-1],
                        alt=group_name,
                        icon=AwesomeIcon(name="plug",
                                         marker_color='green',
                                         icon_color='white'),
                        popup=HTML(value=features["properties"]["name"]))

        m.add_layer(marker)

In [6]:
# 各種オブジェクトの初期化
m = Map(center=[latitude, longitude], zoom=zoom, scroll_wheel_zoom=True)
m.add_control(FullScreenControl())

marker = Marker(location=[latitude, longitude],
                icon=AwesomeIcon(name='car',
                                 marker_color='red',
                                 icon_color='white'))
rect = Rectangle(bounds=gen_bounds(latitude, longitude))
update_fuels(m, rect.bounds, group_name)

# callbackの追加
marker.on_move(marker_callback)
rect.observe(search_callback, "bounds")

m.add_layer(marker)
m.add_layer(rect)

display(m)

Map(center=[35.16597954399553, 136.89914184408428], controls=(ZoomControl(options=['position', 'zoom_in_text',…